# Replication of paper

- "Blended RAG: Improving RAG (Retriever-Augmented Generation) Accuracy with Semantic Search and Hybrid Query-Based Retrievers"
-  Link: [https://arxiv.org/pdf/2404.07220](https://arxiv.org/pdf/2404.07220)

![Figure: Blended RAG Framework](fig.png)


In [1]:
#docker run --rm -p 9200:9200 -p 9300:9300 -e "xpack.security.enabled=false" -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:8.3.3
#docker run --rm -p 9200:9200 -p 9300:9300 -e "xpack.security.enabled=false" -e "discovery.type=single-node" -e "ES_JAVA_OPTS=-Xms512m -Xmx512m" docker.elastic.co/elasticsearch/elasticsearch:8.3.3

In [2]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch("http://localhost:9200")
#es = Elasticsearch("http://localhost:9200",ca_certs=False, verify_certs=False)
es_client.info().body

{'name': '57e3903a6be3',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'R47IQG9vTU6DFlunQhzOuw',
 'version': {'number': '8.3.3',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '801fed82df74dbe537f89b71b098ccaff88d2c56',
  'build_date': '2022-07-23T19:30:09.227964828Z',
  'build_snapshot': False,
  'lucene_version': '9.2.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [3]:
!curl -X GET "localhost:9200/_cluster/health?pretty"

{
  "cluster_name" : "docker-cluster",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 1,
  "number_of_data_nodes" : 1,
  "active_primary_shards" : 3,
  "active_shards" : 3,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


In [4]:
import os
import re
from datetime import date
import pandas as pd
import json
from datetime import datetime
import requests

from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import RequestError

-  Mapping contents (bm25.txt, knn.txt and sparse_encoder.txt) are not in valid JSON formats and we need to   convert to valid JSON format to be used with Elasticsearch index mapping.

-  The content appears to be a Python dictionary, not a JSON object. 
-  JSON requires double quotes for string values and keys, and it does not support comments or trailing commas.

    -- all keys and string values are wrapped in double quotes.    
    -- all Python-specific syntax such as comments or trailing commas are removed.
    

- The updated versions of bm25.txt, knn.txt and sparse_encoder.txt are located  'Input/mapping' path

In [5]:

from sentence_transformers import SentenceTransformer

# Download model for KNN
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

## get the files from specific folder
def get_all_files(folder_name):
    file_path_list = []
    for root, dirs, files in os.walk(folder_name):
        for file in files:
            print(file)
            file_path = os.path.join(root, file)
            file_path_list.append(file_path)
    return file_path_list

## create the index
def create_index(index_name, mapping):
    try:
        es_client.indices.create(index=index_name, body=mapping)
        print(f"Index '{index_name}' created successfully.")
    except RequestError as e:
        if e.error == 'resource_already_exists_exception':
            print(f"Index '{index_name}' already exists.")
        else:
            print(f"An error occurred while creating index '{index_name}': {e}")

def index_data(df_docs, source, index_name, index_name_knn):
    i = 0
    for index, row in df_docs.iterrows():
        i = i + 1
        print("Processing i", i)
        id_ = row['id']
        text = row['document']  # Use 'document' instead of 'text'
        title = row['id']  # Use 'id' as the title or change accordingly
        source = source
        text_embedding = model.encode(text)
        doc = {
            "id": "" + title + "",
            "source": "" + source + "",
            "text_field": "" + text + "",
            "title": "" + title + "",
            "metadata": ""
        }
        doc_knn = {
            "id": "" + title + "",
            "source": "" + source + "",
            "text": "" + text + "",
            "title": "" + title + "",
            "metadata": "",
            "text_embedding": text_embedding
        }
        response = es_client.index(index=index_name, body=doc)
        print(response)
        response = es_client.index(index=index_name_knn, body=doc_knn)
        print(response)


In [6]:
## Example Index name
index_name_knn = 'research_index_knn'
index_name = "research_index_bm25"
index_name_elser = 'research_index_elser'

## Create Index BM25
with open('./input/mapping/bm25.txt', 'r') as file:
    mapping = file.read().rstrip()
create_index(index_name, mapping)

## Create Index Knn
with open('./input/mapping/knn.txt', 'r') as file:
    mapping = file.read().rstrip()
create_index(index_name_knn, mapping)

## Create Index Sparse Encoder for ELSER V1 model
with open('./input/mapping/sparse_encoder.txt', 'r') as file:
    mapping = file.read().rstrip()
create_index(index_name_elser, mapping)


Index 'research_index_bm25' already exists.
Index 'research_index_knn' already exists.
Index 'research_index_elser' already exists.


In [7]:
!curl -X GET "localhost:9200/_cluster/health?pretty"

{
  "cluster_name" : "docker-cluster",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 1,
  "number_of_data_nodes" : 1,
  "active_primary_shards" : 3,
  "active_shards" : 3,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


In [8]:
#In the case of "status" : "red"
# try this:
        
'''
curl -X PUT "localhost:9200/_cluster/settings" -H 'Content-Type: application/json' -d'
{
  "transient": {
    "cluster.routing.allocation.disk.watermark.low": "95%",
    "cluster.routing.allocation.disk.watermark.high": "97%",
    "cluster.routing.allocation.disk.watermark.flood_stage": "98%"
  }
}'
'''    

'\ncurl -X PUT "localhost:9200/_cluster/settings" -H \'Content-Type: application/json\' -d\'\n{\n  "transient": {\n    "cluster.routing.allocation.disk.watermark.low": "95%",\n    "cluster.routing.allocation.disk.watermark.high": "97%",\n    "cluster.routing.allocation.disk.watermark.flood_stage": "98%"\n  }\n}\'\n'

In [9]:
from datasets import load_dataset
# Load the SQuAD dataset (or any other small QA dataset)
ds = load_dataset("squad", split='train[:1000]')

# Extract questions and documents from the dataset
questions = []
docs = []

for example in ds:
    question = example['question']
    document = example['context']
    questions.append([example['id'], question])
    docs.append([example['id'], document])

# Convert to DataFrames
df_question = pd.DataFrame(questions, columns=['id', 'question'])
df_docs = pd.DataFrame(docs, columns=['id', 'document'])


print("Questions DataFrame:")
df_question.head()




Found cached dataset parquet (/home/user/.cache/huggingface/datasets/parquet/plain_text-7d14fdbf55ecc00c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


Questions DataFrame:


,id,question
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...
3,5733be284776f41900661181,What is the Grotto at Notre Dame?
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...


In [10]:
print("\nDocuments DataFrame:")
df_docs.head()



Documents DataFrame:


,id,document
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha..."
1,5733be284776f4190066117f,"Architecturally, the school has a Catholic cha..."
2,5733be284776f41900661180,"Architecturally, the school has a Catholic cha..."
3,5733be284776f41900661181,"Architecturally, the school has a Catholic cha..."
4,5733be284776f4190066117e,"Architecturally, the school has a Catholic cha..."


In [11]:
# research_index_bm25 & research_index_elser (Elasticsearch)
# Define the source name for the data 

source = "squad"

index_data(df_docs, source, index_name, index_name_elser)

Processing i 1
{'_index': 'research_index_bm25', '_id': 'WBsUUJABTUob73jWY8s-', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6739, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'WRsUUJABTUob73jWY8tF', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2000, '_primary_term': 1}
Processing i 2
{'_index': 'research_index_bm25', '_id': 'WhsUUJABTUob73jWY8t3', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6740, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'WxsUUJABTUob73jWY8t-', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2001, '_primary_term': 1}
Processing i 3
{'_index': 'research_index_bm25', '_id': 'XBsUUJABTUob73jWY8ux', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6741, '_primar

{'_index': 'research_index_elser', '_id': 'hRsUUJABTUob73jWaMs6', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2022, '_primary_term': 1}
Processing i 24
{'_index': 'research_index_bm25', '_id': 'hhsUUJABTUob73jWaMtn', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6762, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'hxsUUJABTUob73jWaMtt', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2023, '_primary_term': 1}
Processing i 25
{'_index': 'research_index_bm25', '_id': 'iBsUUJABTUob73jWaMui', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6763, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'iRsUUJABTUob73jWaMuo', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2024, '_primary_term': 1}


{'_index': 'research_index_bm25', '_id': 'sBsUUJABTUob73jWbMt9', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6783, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'sRsUUJABTUob73jWbMuD', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2044, '_primary_term': 1}
Processing i 46
{'_index': 'research_index_bm25', '_id': 'shsUUJABTUob73jWbMu1', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6784, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'sxsUUJABTUob73jWbMu6', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2045, '_primary_term': 1}
Processing i 47
{'_index': 'research_index_bm25', '_id': 'tBsUUJABTUob73jWbMvs', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6785, '_primary_term': 1}
{

{'_index': 'research_index_bm25', '_id': '4BsUUJABTUob73jWccsv', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6807, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': '4RsUUJABTUob73jWccs0', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2068, '_primary_term': 1}
Processing i 70
{'_index': 'research_index_bm25', '_id': '4hsUUJABTUob73jWcctl', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6808, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': '4xsUUJABTUob73jWcctq', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2069, '_primary_term': 1}
Processing i 71
{'_index': 'research_index_bm25', '_id': '5BsUUJABTUob73jWccuN', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6809, '_primary_term': 1}
{

{'_index': 'research_index_elser', '_id': 'CxsUUJABTUob73jWdczG', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2089, '_primary_term': 1}
Processing i 91
{'_index': 'research_index_bm25', '_id': 'DBsUUJABTUob73jWdcz-', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6829, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'DRsUUJABTUob73jWdswD', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2090, '_primary_term': 1}
Processing i 92
{'_index': 'research_index_bm25', '_id': 'DhsUUJABTUob73jWdswt', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6830, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'DxsUUJABTUob73jWdswy', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2091, '_primary_term': 1}


...

{'_index': 'research_index_bm25', '_id': 'eBsVUJABTUob73jWHtKh', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7651, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'eRsVUJABTUob73jWHtKn', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2912, '_primary_term': 1}
Processing i 914
{'_index': 'research_index_bm25', '_id': 'ehsVUJABTUob73jWHtLV', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7652, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'exsVUJABTUob73jWHtLb', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2913, '_primary_term': 1}
Processing i 915
{'_index': 'research_index_bm25', '_id': 'fBsVUJABTUob73jWH9II', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7653, '_primary_term': 1}

{'_index': 'research_index_bm25', '_id': 'pBsVUJABTUob73jWI9LB', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7673, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'pRsVUJABTUob73jWI9LI', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2934, '_primary_term': 1}
Processing i 936
{'_index': 'research_index_bm25', '_id': 'phsVUJABTUob73jWJNIB', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7674, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'pxsVUJABTUob73jWJNIH', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2935, '_primary_term': 1}
Processing i 937
{'_index': 'research_index_bm25', '_id': 'qBsVUJABTUob73jWJNJB', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7675, '_primary_term': 1}

{'_index': 'research_index_bm25', '_id': 'zhsVUJABTUob73jWKNJB', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7694, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'zxsVUJABTUob73jWKNJI', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2955, '_primary_term': 1}
Processing i 957
{'_index': 'research_index_bm25', '_id': '0BsVUJABTUob73jWKNJ5', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7695, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': '0RsVUJABTUob73jWKNJ_', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2956, '_primary_term': 1}
Processing i 958
{'_index': 'research_index_bm25', '_id': '0hsVUJABTUob73jWKNLD', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7696, '_primary_term': 1}

{'_index': 'research_index_elser', '_id': '-xsVUJABTUob73jWLtJY', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2977, '_primary_term': 1}
Processing i 979
{'_index': 'research_index_bm25', '_id': '_BsVUJABTUob73jWLtKa', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7717, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': '_RsVUJABTUob73jWLtKg', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2978, '_primary_term': 1}
Processing i 980
{'_index': 'research_index_bm25', '_id': '_hsVUJABTUob73jWLtLf', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7718, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': '_xsVUJABTUob73jWLtLm', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2979, '_primary_term': 1

{'_index': 'research_index_bm25', '_id': 'JBsVUJABTUob73jWM9O8', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7737, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'JRsVUJABTUob73jWM9PC', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2998, '_primary_term': 1}
Processing i 1000
{'_index': 'research_index_bm25', '_id': 'JhsVUJABTUob73jWM9Pe', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7738, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'JxsVUJABTUob73jWM9Pl', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2999, '_primary_term': 1}


In [13]:
#research_index_knn & research_index_elser (Elasticsearch)
# Define the source name for the data 

source = "squad"

index_data(df_docs, source, index_name_knn, index_name_elser)

Processing i 1
{'_index': 'research_index_knn', '_id': 'KBslUJABTUob73jWPdO-', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4738, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'KRslUJABTUob73jWPdPx', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3000, '_primary_term': 1}
Processing i 2
{'_index': 'research_index_knn', '_id': 'KhslUJABTUob73jWPtMu', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4739, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'KxslUJABTUob73jWPtM0', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3001, '_primary_term': 1}
Processing i 3
{'_index': 'research_index_knn', '_id': 'LBslUJABTUob73jWPtNk', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4740, '_primary_t

{'_index': 'research_index_knn', '_id': 'UhslUJABTUob73jWQdP-', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4759, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'UxslUJABTUob73jWQtMH', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3021, '_primary_term': 1}
Processing i 23
{'_index': 'research_index_knn', '_id': 'VBslUJABTUob73jWQtM2', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4760, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'VRslUJABTUob73jWQtM8', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3022, '_primary_term': 1}
Processing i 24
{'_index': 'research_index_knn', '_id': 'VhslUJABTUob73jWQtNk', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4761, '_primary_term': 1}
{'_i

{'_index': 'research_index_knn', '_id': 'gBslUJABTUob73jWR9OC', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4782, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'gRslUJABTUob73jWR9OH', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3044, '_primary_term': 1}
Processing i 46
{'_index': 'research_index_knn', '_id': 'ghslUJABTUob73jWR9PM', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4783, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'gxslUJABTUob73jWR9PR', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3045, '_primary_term': 1}
Processing i 47
{'_index': 'research_index_knn', '_id': 'hBslUJABTUob73jWSNMG', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4784, '_primary_term': 1}
{'_i

{'_index': 'research_index_elser', '_id': 'rRslUJABTUob73jWTNPA', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3066, '_primary_term': 1}
Processing i 68
{'_index': 'research_index_knn', '_id': 'rhslUJABTUob73jWTNPo', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4805, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'rxslUJABTUob73jWTNPt', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3067, '_primary_term': 1}
Processing i 69
{'_index': 'research_index_knn', '_id': 'sBslUJABTUob73jWTdMf', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4806, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'sRslUJABTUob73jWTdMk', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3068, '_primary_term': 1}
Pr

{'_index': 'research_index_knn', '_id': '1hslUJABTUob73jWUtMt', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4825, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': '1xslUJABTUob73jWUtMz', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3087, '_primary_term': 1}
Processing i 89
{'_index': 'research_index_knn', '_id': '2BslUJABTUob73jWUtNu', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4826, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': '2RslUJABTUob73jWUtNz', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3088, '_primary_term': 1}
Processing i 90
{'_index': 'research_index_knn', '_id': '2hslUJABTUob73jWUtOe', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4827, '_primary_term': 1}
{'_i

...

{'_index': 'research_index_knn', '_id': 'PBsmUJABTUob73jWBNoD', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5644, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'PRsmUJABTUob73jWBNoK', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3906, '_primary_term': 1}
Processing i 908
{'_index': 'research_index_knn', '_id': 'PhsmUJABTUob73jWBNpa', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5645, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'PxsmUJABTUob73jWBNpg', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3907, '_primary_term': 1}
Processing i 909
{'_index': 'research_index_knn', '_id': 'QBsmUJABTUob73jWBNqj', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5646, '_primary_term': 1}
{'

{'_index': 'research_index_knn', '_id': 'ZhsmUJABTUob73jWCNpW', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5665, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'ZxsmUJABTUob73jWCNpf', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3927, '_primary_term': 1}
Processing i 929
{'_index': 'research_index_knn', '_id': 'aBsmUJABTUob73jWCNqU', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5666, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'aRsmUJABTUob73jWCNqZ', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3928, '_primary_term': 1}
Processing i 930
{'_index': 'research_index_knn', '_id': 'ahsmUJABTUob73jWCNq-', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5667, '_primary_term': 1}
{'

{'_index': 'research_index_knn', '_id': 'lhsmUJABTUob73jWDNqD', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5689, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'lxsmUJABTUob73jWDNqK', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3951, '_primary_term': 1}
Processing i 953
{'_index': 'research_index_knn', '_id': 'mBsmUJABTUob73jWDNqr', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5690, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'mRsmUJABTUob73jWDNqw', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3952, '_primary_term': 1}
Processing i 954
{'_index': 'research_index_knn', '_id': 'mhsmUJABTUob73jWDNrP', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5691, '_primary_term': 1}
{'

{'_index': 'research_index_knn', '_id': 'xhsmUJABTUob73jWEdqT', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5713, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'xxsmUJABTUob73jWEdqa', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3975, '_primary_term': 1}
Processing i 977
{'_index': 'research_index_knn', '_id': 'yBsmUJABTUob73jWEtoK', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5714, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': 'yRsmUJABTUob73jWEtoQ', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3976, '_primary_term': 1}
Processing i 978
{'_index': 'research_index_knn', '_id': 'yhsmUJABTUob73jWEtpG', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5715, '_primary_term': 1}
{'

{'_index': 'research_index_knn', '_id': '9hsmUJABTUob73jWFtoD', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5737, '_primary_term': 1}
{'_index': 'research_index_elser', '_id': '9xsmUJABTUob73jWFtoI', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 3999, '_primary_term': 1}
